In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from zipfile import ZipFile
from PIL import Image
from sklearn.metrics import accuracy_score

In [8]:
# with ZipFile('Fashion-MNIST.zip', 'r') as zip_ref:
#     zip_ref.extractall('fashion_mnist')

# Load class mapping
class_mapping = pd.read_csv('Fashion-MNIST/classes.tsv', sep='\t', header=None)
class_dict = {i: label for i, label in enumerate(class_mapping[0])}


# Load training labels
gt_train = pd.read_csv('Fashion-MNIST/gt-train.tsv', sep='\t', header=None, names=['file', 'label'])




In [10]:
# Prepare training data
train_images = []
train_labels = []

for idx, row in gt_train.iterrows():
    img_path = os.path.join('Fashion-MNIST', row['file'])
    img = Image.open(img_path).convert('L').resize((28, 28))
    train_images.append(np.array(img))
    train_labels.append(row['label'])

x_train = np.array(train_images).reshape(-1, 28, 28, 1) / 255.0
y_train = to_categorical(train_labels, num_classes=len(class_dict))

# Prepare testing data
test_images = []
test_paths = []

for test_file in sorted(os.listdir('Fashion-MNIST/test')):
    img_path = os.path.join('Fashion-MNIST/test', test_file)
    img = Image.open(img_path).convert('L').resize((28, 28))
    test_images.append(np.array(img))
    test_paths.append(f"test/{test_file}")

x_test = np.array(test_images).reshape(-1, 28, 28, 1) / 255.0

In [12]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(class_dict), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\alan8\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.1)


Epoch 1/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.8450 - loss: 0.4302 - val_accuracy: 0.0000e+00 - val_loss: 14.6179
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.8747 - loss: 0.3397 - val_accuracy: 0.0000e+00 - val_loss: 17.1819
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.8907 - loss: 0.2970 - val_accuracy: 0.0000e+00 - val_loss: 17.6600
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.9005 - loss: 0.2691 - val_accuracy: 0.0000e+00 - val_loss: 20.3326
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.9116 - loss: 0.2418 - val_accuracy: 0.0000e+00 - val_loss: 22.3894
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.9164 - loss: 0.2255 - val_accuracy: 0.0000e+00 - val_loss: 24.3686
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.9229 - loss: 0.2049 - val_accuracy: 0.0000e+00 - val_loss: 22.6910
Epoch 8/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step -

In [16]:
predictions = np.argmax(model.predict(x_test), axis=1)



313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [17]:
submission_df = pd.DataFrame({
    'file': test_paths,
    'prediction': predictions
})
submission_df.to_csv('gt-test.tsv', sep='\t', index=False)

print("Submission file gt-test.tsv created successfully.")

Submission file gt-test.tsv created successfully.
